In [1]:
# coin toss combos

# Say we flip a coin n times and we get paid $1 for each H greater than some value k the formula below is the combinatorics formula

import math

def combos(n, i):
    return math.factorial(n) / (math.factorial(n-i)*math.factorial(i))

for i in range(5):
    print(combos(4,i))

1.0
4.0
6.0
4.0
1.0


In [2]:
# Fair value
# 

fair_value = 0 
n= 4 # number of coin flips
for k in range(n+1):
    fair_value += combos(n,k)*0.5**k*0.5**(n-k) * k
    
print(fair_value)

2.0


In [3]:
# Now consider if we wanted to price a game in which we get $1 per head over a certain threshold. 
# Let's take an example where we flip a coin 10 times and get $1 for each head over 6.

fair_value = 0 
n= 10 # number of coin flips
for k in range(7,n+1):
    fair_value += combos(n,k)*0.5**k*0.5**(n-k) * k
    
print(fair_value)

1.26953125


__Option Pricing__

In [4]:
import numpy as np
# Initialise parameters
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.06      # annual risk-free rate
N = 3         # number of time steps
u = 1.1       # up-factor in binomial models
d = 1/u       # ensure recombining tree
opttype = 'C' # Option Type 'C' or 'P'

In [6]:
def binomial_tree_slow(K,T,S0,r,N,u,d,opttype='C'):
    #precompute constants
    dt = T/N
    q = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)
    
    # initialise asset prices at maturity - Time step N
    S = np.zeros(N+1)
    S[0] = S0*d**N
    for j in range(1,N+1):
        S[j] = S[j-1]*u/d
    
    # initialise option values at maturity
    C = np.zeros(N+1)
    for j in range(0,N+1):
        C[j] = max(0, S[j]-K)
        
    # step backwards through tree
    for i in np.arange(N,0,-1):
        for j in range(0,i):
            C[j] = disc * ( q*C[j+1] + (1-q)*C[j] )
    
    return C[0]

binomial_tree_slow(K,T,S0,r,N,u,d,opttype='C')

10.145735799928817

In [7]:

def binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C'):
    #precompute constants
    dt = T/N
    q = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)
    
    # initialise asset prices at maturity - Time step N
    C = S0 * d ** (np.arange(N,-1,-1)) * u ** (np.arange(0,N+1,1)) 
    
    # initialise option values at maturity
    C = np.maximum( C - K , np.zeros(N+1) )
        
    # step backwards through tree
    for i in np.arange(N,0,-1):
        C = disc * ( q * C[1:i+1] + (1-q) * C[0:i] )
    
    return C[0]

binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C')

10.145735799928826

In [8]:
for N in [3,50, 100, 1000, 5000]:
    binomial_tree_slow(K,T,S0,r,N,u,d,opttype='C')
    binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C')